#### Tecnologias Utilizadas:

- Python 
- SageMath


# El Gammal (PKE)


**Public Key Encryption** é um tipo de criptografia assimétrica que utiliza um par de chaves: uma chave pública para cifrar mensagens e uma chave privada para decifrá-las. A principal característica do PKE é que a chave pública pode ser compartilhada abertamente, enquanto a chave privada deve ser mantida em segredo.

Com PKEs deixa de ser necessário um canal para partilhar chaves, já que basta cifrar algo com a chave publica do destinatário, o que garante que este será o único a conseguir decifrá-la.

### **1ª Etapa**

Começamos por gerar os parametros que seguem as restrições estabelecidas no enunciado:

#### **Parameteros**

- **Parametro de segurança (λ) = 128**

```py
        _lambda = 128  # security parameter
        lambda_bits = _lambda.nbits() # 128 in binary is 10000000 -> 8 bits
```
Quanto maior lambda, mais seguro o sistema, mas em aplicações modernas lambda = 128 é o mais usual.

- **p : numero primo que define o grupo multiplicativo ℤₚ***
```py   
        while True:
            k = randint(min_k, max_k)
            p_candidate = k * q + 1
            if is_prime(p_candidate) and p_candidate.nbits() >= min_p_bits:
                p = p_candidate
                break
            k += 1
```
Isto significa que teremos um grupo com numeros de 1 a p-1, sendo este ultimo divisivel por q.
Segundo o enunciado este terá ∣p∣≥λ×∣λ∣​ que neste caso serão 1024 bits.

- **q : numero primo que define a ordem do subgrupo**
```py 
        lower_bound = 2^(_lambda - 1)
        upper_bound = 2^_lambda - 1
        q_candidate = randint(lower_bound, upper_bound)
        q = next_prime(q_candidate)
```
Garante a existencia do subgrupo e afeta diretamente a segurança do sistema: pois quanto maior lambda, maior q e maior é a resistencia do sistema contra ataques de logaritmo discreto.

- **g : gerador do subgrupo de ordem g**

```py
        while True:
            a = randint(2, p - 2)
            g = power_mod(a, (p - 1) // q, p)
            if g != 1:
                break
```
O conjunto {g¹, g², g³, ..., g^q} gera todos os elementos do subgrupo

#### **Par de Chaves**

- **s : chave privada**

```py
        s = randint(1, q-1)
```

Numero aleatório entre 1 e q-1 que é mantido em segredo pelo proprietário e só usado para decifrar

- **h : chave publica**

```py
        h = power_mod(g, s, p)
```
Esta chave é usada para cifrar as mensagens. É muito dificil chegar a s a partir de h (problema do logaritmo discreto)

#### **Ao cifrar**


